### In this notebook we'll try to evaluate VGG thanks to our abstract engine

In [4]:
%pip install memory_profiler


Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
sys.path.append('../src')
sys.path.append('../src')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import vgg19
from abstractModule import AbstractLinear as AL
from abstractModule import AbstractReLU as AR
from abstractModule import AbstractMaxpool2D as AM
device=torch.device("cpu")
%load_ext autoreload
%autoreload 2

In [2]:
model = vgg19(pretrained=True)
model

/home/guillaume/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/guillaume/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [4]:
import torch
import torch.nn as nn

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        
        # Couches de convolution
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.conv7 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.conv9 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.conv10 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv11 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv12 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv13 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv14 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv15 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv16 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        
        # Couches de ReLU
        self.relu = nn.ReLU(inplace=True)
        
        # Couches de pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Couches Fully Connected (FC)
        self.fc1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(25088, 4096),
        
        )
        
        self.fc2 = nn.Sequential(nn.Flatten(),
            nn.Linear(4096, 4096),
        )
        
        self.fc3 = nn.Sequential(nn.Flatten(),nn.Linear(4096, 1000))
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        x = self.pool(x)
        
        x = self.conv3(x)
        x = self.relu(x) 
        x = self.conv4(x)
        x = self.relu(x)

        x = self.pool(x)

        x = self.conv5(x)
        x = self.relu(x)
        x = self.conv6(x)
        x = self.relu(x)
        x = self.conv7(x)
        x = self.relu(x)
        x = self.conv8(x)
        x = self.relu(x)

        x = self.pool(x)

        x = self.conv9(x)
        x = self.relu(x)
        x = self.conv10(x)
        x = self.relu(x)
        x = self.conv11(x)
        x = self.relu(x)
        x = self.conv12(x)
        x = self.relu(x)

        x = self.pool(x)

        x = self.conv13(x)
        x = self.relu(x)
        x = self.conv14(x)
        x = self.relu(x)
        x = self.conv15(x)
        x = self.relu(x)
        x = self.conv16(x)
        x = self.relu(x)

        x = self.pool(x)

 
        
        
        # Fully Connected Layers
        x = self.fc1(x)
        x= self.relu(x)
        x = self.fc2(x)
        x= self.relu(x)
        x = self.fc3(x)
        x= self.relu(x)
        
        return x
    
    def abstract_forward(self,x,add_symbol=False,device=torch.device("cpu")):
        self.device=device
        
        x_true = x
        x_true = x_true[0].unsqueeze(0)

        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv1,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)


        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv2,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      

        x,x_min,x_max ,x_true = AM.abstract_maxpool2D(self.pool,x,x_true,add_symbol=add_symbol,device=self.device)
        
        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv3,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      

        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv4,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      


        x,x_min,x_max ,x_true = AM.abstract_maxpool2D(self.pool,x,x_true,add_symbol=add_symbol,device=self.device)

        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv5,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      

        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv6,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      

        
        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv7,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      

        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv8,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      


        x,x_min,x_max ,x_true = AM.abstract_maxpool2D(self.pool,x,x_true,add_symbol=add_symbol,device=self.device)

        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv9,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      

        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv10,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      

        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv11,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      

        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv12,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      


        x,x_min,x_max ,x_true = AM.abstract_maxpool2D(self.pool,x,x_true,add_symbol=add_symbol,device=self.device)

        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv13,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      

        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv14,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      

        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv15,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      

        x,x_min,x_max,x_true = AL.abstract_conv2D(self.conv16,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu_conv2D(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      


        x,x_min,x_max ,x_true = AM.abstract_maxpool2D(self.pool,x,x_true,add_symbol=add_symbol,device=self.device)

 
        
        
        # Fully Connected Layers
        x,x_min,x_max,x_true = AL.abstract_linear(self.fc1,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
        
        x,x_min,x_max,x_true = AL.abstract_linear(self.fc2,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
        
        
        x,x_min,x_max,x_true = AL.abstract_linear(self.fc3,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
        return x,x_min,x_max,x_true


In [5]:
C=CustomCNN()
C = C.to(device)

In [6]:
for param_1, param_2 in zip(C.parameters(), model.parameters()):
    print(param_1.data.shape, param_2.data.shape)
    param_1.data = param_2.data.clone()

torch.Size([64, 3, 3, 3]) torch.Size([64, 3, 3, 3])
torch.Size([64]) torch.Size([64])
torch.Size([64, 64, 3, 3]) torch.Size([64, 64, 3, 3])
torch.Size([64]) torch.Size([64])
torch.Size([128, 64, 3, 3]) torch.Size([128, 64, 3, 3])
torch.Size([128]) torch.Size([128])
torch.Size([128, 128, 3, 3]) torch.Size([128, 128, 3, 3])
torch.Size([128]) torch.Size([128])
torch.Size([256, 128, 3, 3]) torch.Size([256, 128, 3, 3])
torch.Size([256]) torch.Size([256])
torch.Size([256, 256, 3, 3]) torch.Size([256, 256, 3, 3])
torch.Size([256]) torch.Size([256])
torch.Size([256, 256, 3, 3]) torch.Size([256, 256, 3, 3])
torch.Size([256]) torch.Size([256])
torch.Size([256, 256, 3, 3]) torch.Size([256, 256, 3, 3])
torch.Size([256]) torch.Size([256])
torch.Size([512, 256, 3, 3]) torch.Size([512, 256, 3, 3])
torch.Size([512]) torch.Size([512])
torch.Size([512, 512, 3, 3]) torch.Size([512, 512, 3, 3])
torch.Size([512]) torch.Size([512])
torch.Size([512, 512, 3, 3]) torch.Size([512, 512, 3, 3])
torch.Size([512]) 

In [7]:
x=torch.randn(1,3,224,224)
print(torch.sum(C(x)-model(x)))

tensor(619.7573, grad_fn=<SumBackward0>)


In [9]:
from abstract import abstractTensor as AT 
x=torch.randn(3,224,224)
x=AT(x,alpha=0.01*torch.ones(200)).abstract_tensor()

In [ ]:
x=x.to(device)
C.eval()

with torch.no_grad():
   result,x_min,x_max,x_true= C.abstract_forward(x,add_symbol=True,device=device)



In [18]:
_ , index = torch.topk(result[0], 8)


print(f"y_min       =  {x_min[index]}")
print(f"y_max       =  {x_max[index]}")
print(f"center Ztp  =  {result[0][index]}")
print(f"y_true      =  {x_true.squeeze(0)[index]}")
print(f"y_max-x_min =  {x_max[index]-x_min[index]}")
print(f"Trash symbol=  {result[-1][index]}")

y_min       =  tensor([5.2149, 5.0988, 5.0232, 4.9996, 4.9369, 4.4662, 4.3951, 4.3569])
y_max       =  tensor([5.2298, 5.1098, 5.0322, 5.0107, 4.9469, 4.4785, 4.4089, 4.3711])
center Ztp  =  tensor([5.2223, 5.1043, 5.0277, 5.0051, 4.9419, 4.4723, 4.4020, 4.3640])
y_true      =  tensor([5.2222, 5.1044, 5.0279, 5.0052, 4.9420, 4.4723, 4.4020, 4.3639])
y_max-x_min =  tensor([0.0149, 0.0110, 0.0089, 0.0111, 0.0100, 0.0123, 0.0138, 0.0143])
Trash symbol=  tensor([0., 0., 0., 0., 0., 0., 0., 0.])
